# PREPARAÇÃO
ATENÇÂO: Lembrar de alterar o tipo de execução para R.

In [ ]:
# Limpando Environment e Memória ===============================================
rm(list=ls(all=T))
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658395,35.2,1454438,77.7,1335218,71.4
Vcells,1225330,9.4,8388608,64.0,1975093,15.1


In [ ]:
# Bibliotecas ==================================================================
# Instala e carrega pacman (para facilitar o gerenciamento de pacotes)
if (!requireNamespace("pacman", quietly = TRUE)) install.packages("pacman")

p_load(
  tidyverse,           # Conjunto de pacotes para manipulação e visualização de dados (dplyr, ggplot2, tidyr, purrr, stringr, lubridate etc)
  plyr,                # Para usar o mapvalues
  data.table,          # Manipulação eficiente e rápida de grandes tabelas de dados
  janitor,             # Limpeza e organização de dados
  openxlsx,            # Leitura e escrita de arquivos Excel (xlsx) sem necessidade do Excel instalado
  readxl,              # Leitura de arquivos Excel (xls e xlsx)
  rio,                 # Importação e exportação simplificada de vários formatos de arquivo
  skimr,               # Resumo rápido e completo das variáveis do dataframe
  vroom,               # Leitura rápida de arquivos CSV e similares, otimizando desempenho
  arrow,               # Interface para arquivos Apache Arrow e Parquet para alta performance em I/O
  googleCloudStorageR  # Interface para Google Cloud Storage, upload/download de arquivos na nuvem
  )

In [ ]:
# Acesso e autenticação ao GCS no R no Colab ===================================
# (subir arquivo da chave no ambiente antes)
chave_jason <- "<sua-chave-json-aqui>"

Sys.setenv("GCS_AUTH_FILE" = chave_jason)
Sys.setenv("GAR_CLIENT_JSON" = chave_jason)

gcs_auth(json_file = chave_jason)

In [ ]:
# Ajustando diretórios =========================================================
bucket <- "<seu-bucket-json-aqui"
prefixo_brutos <- "<seu-prefixo-json-aqui"
prefixo_processados <- "<seu-prefixo-json-aqui"

In [ ]:
# Abrindo Dicionários (subir arquivo do dicionário no ambiente antes) ==========
dicionarios <- lapply(
  excel_sheets("dicionários.xlsx"),
  function(sheet) {
    read_excel("dicionários.xlsx", sheet = sheet)
  }
)

dic_classes <- dicionarios[[1]]
dic_faixaeta <- dicionarios[[2]]
dic_escolaridade <- dicionarios[[3]]
dic_sexo <- dicionarios[[4]]
dic_racacor <- dicionarios[[5]]
dic_local <- dicionarios[[6]]

rm(dicionarios)

# PROCESSAMENTO

In [ ]:
# Funções ======================================================================
# Parâmetros -------------------------------------------------------------------
salario_minimo <- c(
  '2020' = 1045,
  '2021' = 1100,
  '2022' = 1212,
  '2023' = 1302,
  '2024' = 1412,
  '2025' = 1518
  )

colunas_selecionadas <- c(
  "competenciamov", "regiao", "uf",
  "subclasse","sexo", "idade", "graudeinstrucao", "racacor", "salario",
  "admissoes", "desligamentos", "saldomovimentacao"
  )

colunas_agrupar <- c(
  "competenciamov", "regiao", "uf",
  "subclasse","sexo", "idade", "graudeinstrucao", "racacor", "salario"
  )

# filtro_uf <- 31

# Funções auxiliares -----------------------------------------------------------
adicionar_faixaeta <- function(idade) {
  case_when(
    idade >= 10 & idade <= 14 ~ 1,
    idade >= 15 & idade <= 17 ~ 2,
    idade >= 18 & idade <= 24 ~ 3,
    idade >= 25 & idade <= 29 ~ 4,
    idade >= 30 & idade <= 39 ~ 5,
    idade >= 40 & idade <= 49 ~ 6,
    idade >= 50 & idade <= 64 ~ 7,
    idade >= 65 ~ 8,
    TRUE ~ 99
  )
}

adicionar_regiao <- function(regiao) {
  case_when(
    regiao == 1 ~ "Norte",
    regiao == 2 ~ "Nordeste",
    regiao == 3 ~ "Sudeste",
    regiao == 4 ~ "Sul",
    regiao == 5 ~ "Centro-Oeste",
    TRUE ~ "Não Informado"
  )
}

# Função que realiza o processamento específico dos dados ----------------------
processamento <- function(dados, tipo_arquivo) {

  # Renomeia colunas para minúsculas e remove acentos
  setnames(dados, old = names(dados), new = tolower(stri_trans_general(names(dados), "Latin-ASCII")))

  # Aplica filtro de UF
  # dados <- dados[uf == filtro_uf]

  # Ajusta colunas de admissoes, desligamentos e saldo com base no tipo de arquivo
  if (tipo_arquivo == "CAGEDEXC") {
    dados[, `:=`(
      admissoes = ifelse(saldomovimentacao == 1, -1, 0),
      desligamentos = ifelse(saldomovimentacao == -1, -1, 0),
      saldomovimentacao = ifelse(saldomovimentacao == 1, -1, 1)
    )]
  } else {
    dados[, `:=`(
      admissoes = ifelse(saldomovimentacao == 1, 1, 0),
      desligamentos = ifelse(saldomovimentacao == -1, 1, 0)
    )]
  }

  # Seleciona apenas as colunas necessárias (filtra as que existem no data)
  cols_existentes <- intersect(colunas_selecionadas, names(dados))
  dados <- dados[, ..cols_existentes]

  # Agrupa e soma variáveis numéricas
  dados_agrupados <- dados[, .(
    admissoes = sum(admissoes, na.rm = TRUE),
    desligamentos = sum(desligamentos, na.rm = TRUE),
    saldomovimentacao = sum(saldomovimentacao, na.rm = TRUE)
  ), by = colunas_agrupar]

  # Ajusta ano e converte competenciamov em Date
  dados_agrupados[, ano := substr(competenciamov, 1, 4)]
  dados_agrupados[, competenciamov := suppressWarnings(as.Date(
    paste0(substr(as.character(competenciamov), 1, 6), "01"), format = "%Y%m%d"))]

  # Ajusta salario: troca vírgula por ponto, trata NA como zero
  dados_agrupados[, salario := as.numeric(str_replace_na(str_replace(as.character(salario), ",", "."), "0"))]

  # Salário mínimo do ano
  dados_agrupados[, salario_minimo_ano := salario_minimo[ano]]

  # Faixa salarial com múltiplos níveis
  dados_agrupados[, faixa_salarial := fifelse(
    salario <= salario_minimo_ano, "Até 1 salário mínimo",
    fifelse(salario > salario_minimo_ano & salario <= 2 * salario_minimo_ano, "Entre 1 e 2 salários mínimos",
      fifelse(salario > 2 * salario_minimo_ano & salario <= 3 * salario_minimo_ano, "Entre 2 e 3 salários mínimos",
        fifelse(salario > 3 * salario_minimo_ano & salario <= 4 * salario_minimo_ano, "Entre 3 e 4 salários mínimos",
          fifelse(salario > 4 * salario_minimo_ano & salario <= 5 * salario_minimo_ano, "Entre 4 e 5 salários mínimos",
            fifelse(salario > 5 * salario_minimo_ano & salario <= 10 * salario_minimo_ano, "Entre 5 e 10 salários mínimos",
              "Mais de 10 salários mínimos"
            )
          )
        )
      )
    )
  )]

  # Faixa etária usando função auxiliar
  dados_agrupados[, faixaetaria := adicionar_faixaeta(idade)]

  # Mapeamento dos dicionários usando mapvalues
  dados_agrupados[, sexo := mapvalues(sexo, from = dic_sexo$cod, to = dic_sexo$nom, warn_missing = FALSE)]
  dados_agrupados[, racacor := mapvalues(racacor, from = dic_racacor$cod, to = dic_racacor$nom, warn_missing = FALSE)]
  dados_agrupados[, graudeinstrucao := mapvalues(graudeinstrucao, from = dic_escolaridade$cod, to = dic_escolaridade$nom, warn_missing = FALSE)]
  dados_agrupados[, faixaeta := mapvalues(faixaetaria, from = dic_faixaeta$cod, to = dic_faixaeta$nom, warn_missing = FALSE)]
  dados_agrupados[, subclasse_original := subclasse]
  dados_agrupados[, subclasse := mapvalues(subclasse_original, from = dic_classes$Subclasse, to = dic_classes$`Nome Subclasse`, warn_missing = FALSE)]
  dados_agrupados[, classe := mapvalues(subclasse_original, from = dic_classes$Subclasse, to = dic_classes$`Nome Classe`, warn_missing = FALSE)]
  dados_agrupados[, secao := mapvalues(subclasse_original, from = dic_classes$Subclasse, to = dic_classes$`Nome Seção`, warn_missing = FALSE)]
  dados_agrupados[, uf := mapvalues(uf, from = dic_local$UF, to = dic_local$Nome_UF, warn_missing = FALSE)]

  # Região nomeada via função auxiliar (adicionar_regiao)
  dados_agrupados[, regiao := adicionar_regiao(regiao)]

  # Remover coluna auxiliar ano e subclasse_original se desejar
  dados_agrupados[, ano := NULL]
  dados_agrupados[, subclasse_original := NULL]
  dados_agrupados[, salario_minimo_ano := NULL]
  dados_agrupados[, faixaetaria := NULL]
}

In [ ]:
# Listar arquivos .txt no bucket
arquivos_txt <- gcs_list_objects(bucket = bucket, prefix = prefixo_brutos)$name
arquivos_txt <- arquivos_txt[grepl("\\.txt$", arquivos_txt)]

# Loop de processamento de cada arquivo bruto
for (arquivo in arquivos_txt) {
  cat("\nIniciando processamento do arquivo:", arquivo, "\n")

  # 1. Construir o caminho completo do arquivo no bucket para baixar
  caminho_arquivo_bruto <- arquivo

  # 2. Baixar o arquivo .txt e ler como data.table
  # gcs_get_object retorna o conteúdo do arquivo, passamos para fread via input
  dados <- gcs_get_object(object_name = caminho_arquivo_bruto, bucket = bucket) %>%
    fread(input = ., stringsAsFactors = TRUE, encoding = "UTF-8")

  # 3. Extrai tipo do arquivo (CAGEDEXC, CAGEDMOV2021 etc)
  tipo_arquivo <- sub("(CAGED[A-Z]+).*", "\\1", arquivo)

  # 4. Aplicar o processamento definido
  dados_processados <- processamento(dados, tipo_arquivo)

  # 5. Definir nome do arquivo .parquet para salvar localmente
  arquivo_parquet <- gsub(".txt", ".parquet", basename(arquivo))
  write_parquet(dados_processados, arquivo_parquet)

  # 6. Construir o caminho onde salvar no bucket processed
  caminho_arquivo_processado <- paste0(prefixo_processados, arquivo_parquet)

  # 7. Enviar o arquivo parquet gerado para o bucket processed no GCS
  gcs_upload(
    file = arquivo_parquet,
    bucket = bucket,
    name = caminho_arquivo_processado,
    upload_type = "simple",
    predefinedAcl = "bucketLevel")


  # 8. Remover arquivo parquet local para liberar espaço
  file.remove(arquivo_parquet)
   cat("Arquivo processado e enviado com sucesso:", caminho_arquivo_processado, "\n")
}

cat("\nProcessamento concluído para", length(arquivos_txt), "arquivo(s).\n")